In [2]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict

In [3]:
train_path = "G:\Data Science\Datasets\Credit Risk Model Stability\csv_files/train/"
train_dfs = os.listdir(train_path)
print(train_dfs)
# train_dfs = [train_path + i for i in train_dfs]
print(train_dfs)

['train_applprev_1_0.csv', 'train_applprev_1_1.csv', 'train_applprev_2.csv', 'train_base.csv', 'train_credit_bureau_a_1_0.csv', 'train_credit_bureau_a_1_1.csv', 'train_credit_bureau_a_1_2.csv', 'train_credit_bureau_a_1_3.csv', 'train_credit_bureau_a_2_0.csv', 'train_credit_bureau_a_2_1.csv', 'train_credit_bureau_a_2_10.csv', 'train_credit_bureau_a_2_2.csv', 'train_credit_bureau_a_2_3.csv', 'train_credit_bureau_a_2_4.csv', 'train_credit_bureau_a_2_5.csv', 'train_credit_bureau_a_2_6.csv', 'train_credit_bureau_a_2_7.csv', 'train_credit_bureau_a_2_8.csv', 'train_credit_bureau_a_2_9.csv', 'train_credit_bureau_b_1.csv', 'train_credit_bureau_b_2.csv', 'train_debitcard_1.csv', 'train_deposit_1.csv', 'train_other_1.csv', 'train_person_1.csv', 'train_person_2.csv', 'train_static_0_0.csv', 'train_static_0_1.csv', 'train_static_cb_0.csv', 'train_tax_registry_a_1.csv', 'train_tax_registry_b_1.csv', 'train_tax_registry_c_1.csv']
['train_applprev_1_0.csv', 'train_applprev_1_1.csv', 'train_applprev_2.

In [4]:
data_categories = set([file_name.split("_")[1] for file_name in train_dfs])
print(data_categories)
data = defaultdict(list)
for file_name in train_dfs:
    name_split = file_name.split("_")
    data[name_split[1]].append(train_path + file_name)

def load_data(data_dict,category):
    '''
    Loads data according to the category in a df and stores it in the dictionary

    '''
    df_list = []
    for path in data_dict[category]:    
        temp_df = pd.read_csv(path)
        df_list.append(temp_df)
    data_dict[category] = df_list


{'tax', 'applprev', 'static', 'credit', 'deposit', 'person', 'other', 'base.csv', 'debitcard'}


In [5]:
# base_df contains case id and basic stuff
base_df = pd.read_csv(train_path + "train_base.csv")
# load_data(data,'applprev')
load_data(data,'debitcard')

In [6]:
# important stats on base df
# total cases = 1526659
#
base_df.head()

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1


In [7]:
len(data["debitcard"])

1

In [8]:
def basic_description(df,nulls_threshold = 0.3):
    print("All columns and their null value counts")
    for i in df.columns:
        nulls = df[i].isna().sum()
        if (nulls/df.shape[0]) < nulls_threshold:
            print(i)
            print("null percentage ", nulls/df.shape[0])
            print("number of unique values ", df[i].nunique())
            print(df[i].unique()[:10])
            print()
        else:
            print(i)
            print(f"{i} has too many nulls")
            print()
    
basic_description(data["debitcard"][0],1)


All columns and their null value counts
case_id
null percentage  0.0
number of unique values  111772
[225 331 358 390 445 450 453 582 649 697]

last180dayaveragebalance_704A
null percentage  0.922340466109776
number of unique values  2495
[          nan 0.0000000e+00 2.5145560e+00 6.9115600e-01 1.8276545e+01
 1.7412056e+01 1.1111111e+02 5.0540004e+00 2.3000000e+01 1.6666666e+03]

last180dayturnover_1134A
null percentage  0.9295558861298648
number of unique values  2581
[       nan 5.0000e+04 7.0000e-02 1.3288e+02 1.0000e+05 6.0000e+04
 4.7400e+04 9.0554e+03 3.0000e+04 2.0000e+04]

last30dayturnover_651A
null percentage  0.9295558861298648
number of unique values  482
[       nan      0.    100000.    150000.     64000.     65800.
  13870.     24100.     24224.201  10100.   ]

num_group1
null percentage  0.0
number of unique values  66
[0 1 2 3 4 5 6 7 8 9]

openingdate_857D
null percentage  0.08080634702673838
number of unique values  1578
['2016-08-16' '2015-03-19' '2014-09-02' '2014-

#####  Interesting things about the data
* Appl prev

    * Important columns : actualdpd_943P - days past due,credacc_credlmt_575A,credamount_590A,credtype_587L,currdebt_94A
    district_544M,downpmt_134A,status_219L,tenor_203L
    * Feature creation column - approvaldate_319D,credtype_587L,district_544M,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,tenor_203L

    * childnum_21L - why 20 unique values??

* person
    * Important columns : zipcode,education_927M,empl_industry_691L,gender_992L,mainoccupationinc_384A,type_25L

* debitcard
    * Important columns : 


    


#### Feature engineering ideas 
* Feature around location (zipcode etc)
* Feature around total debt, acc balance etc,
* counts on number of accts, cards, 
* total of tenor 
* date from features



305137